In [1]:
import os
from glob import glob
from skimage.metrics import peak_signal_noise_ratio
import torch
import numpy as np
from PIL import Image

from notebook_helpers import get_model, run, get_cond_for_psnr_testing

c:\Users\oKata\anaconda3\envs\ldm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = 'X:/University/MastersDegree/DeepLearning/data/DIV2K_valid_HR/'
SAVE_PATH = 'X:/University/MastersDegree/DeepLearning/latent-diffusion'
DIFFUSION_STEPS = 100
model = get_model('superresolution')

Using downloaded and verified file: logs\diffusion\superresolution_bsr\configs\project.yaml\dl=1
Using downloaded and verified file: logs\diffusion\superresolution_bsr\checkpoints\last.ckpt\dl=1
Loading model from logs\diffusion\superresolution_bsr\checkpoints\last.ckpt\dl=1
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 113.62 M params.
Keeping EMAs of 308.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [3]:
image_filenames = glob(os.path.join(DATASET_PATH, '*'))

In [5]:
image_filenames = image_filenames[0:1]

In [6]:
def extract_sample(logs) -> Image:
    sample = logs["sample"]
    sample = sample.detach().cpu()
    sample = torch.clamp(sample, -1., 1.)
    sample = (sample + 1.) / 2. * 255
    sample = sample.numpy().astype(np.uint8)
    sample = np.transpose(sample, (0, 2, 3, 1))
    return sample[0]

def estimate_psnr(image_filenames, model, diff_steps, save_path=None):
    psnr_vals = []
    for image_file in image_filenames:
        example = get_cond_for_psnr_testing('superresolution', image_file)
        logs = run(model, example, 'superresolution', diff_steps)
        
        orig_image = np.asarray(example['orig'])
        test_image = extract_sample(logs)
        psnr = peak_signal_noise_ratio(orig_image, test_image)
        psnr_vals.append(psnr)
        
        if save_path is not None:
            image_name = image_file.split(os.sep)[-1]
            savepath = os.path.join(save_path, image_name)
            test_image.save(save_path)
    return np.mean(psnr_vals)

In [7]:
psnr = estimate_psnr(image_filenames, model, DIFFUSION_STEPS, SAVE_PATH)

Plotting: Switched to EMA weights
Sampling with eta = 1.0; steps: 100
Data shape for DDIM sampling is (1, 3, 339, 510), eta 1.0
Running DDIM Sampling with 100 timesteps


DDIM Sampler: 100%|██████████| 100/100 [05:19<00:00,  3.20s/it]


Plotting: Restored training weights


In [8]:
psnr[0].shape

(1, 1356, 2040, 3)

In [9]:
psnr[1].shape

(1356, 2040, 3)

In [11]:
example = get_cond_for_psnr_testing('superresolution', image_filenames[0])

In [12]:
example['LR_image'].shape, example['image'].shape, example['orig'].shape

(torch.Size([1, 3, 64, 64]),
 torch.Size([1, 3, 64, 64]),
 torch.Size([1, 3, 256, 256]))

In [13]:
psnr[1]

array([[[ 24, 142, 208],
        [  6, 151, 226],
        [  0, 149, 228],
        ...,
        [  7, 127, 199],
        [  2, 134, 205],
        [ 33, 116, 168]],

       [[  1, 148, 221],
        [  0, 154, 224],
        [  0, 150, 226],
        ...,
        [  0, 134, 197],
        [  0, 132, 190],
        [  1, 122, 191]],

       [[  1, 150, 216],
        [  0, 160, 214],
        [  0, 156, 215],
        ...,
        [  0, 136, 192],
        [  0, 141, 190],
        [  0, 131, 193]],

       ...,

       [[  0,  88, 128],
        [  0,  95, 130],
        [  0,  94, 128],
        ...,
        [  0,  76,  97],
        [  0,  78, 100],
        [  0,  73, 101]],

       [[  3,  88, 129],
        [  0,  93, 134],
        [  0,  93, 134],
        ...,
        [  0,  75, 100],
        [  0,  74, 103],
        [  0,  73, 101]],

       [[ 23,  85, 119],
        [  5,  89, 126],
        [  0,  91, 126],
        ...,
        [  0,  73,  98],
        [  0,  73,  96],
        [ 12,  73,  92]]

In [14]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'